# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [40]:
import pandas as pd
import dask.dataframe as dd
import random
import itertools
import numpy as np
from datetime import datetime
from tqdm import tqdm
import nltk
import string
from nltk.corpus import stopwords
import fasttext
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importation des données

In [41]:
DBRaw = dd.read_parquet('../data/extraction_sirene_20220510.parquet', engine='pyarrow')

Temporaire : On restreint les données à 1% de la base initiales.

In [42]:
DBRaw = DBRaw.sample(frac= 0.01, random_state=1234)

On transforme les valeurs manquantes en NaN.

In [43]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [44]:
Var2Keep = ["APE_SICORE","LIB_SICORE","AUTO","DATE","NAT_SICORE","SED_SICORE","EVT_SICORE","SURF"]
DB = DBRaw[Var2Keep]

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [45]:
DB = DB.dropna(subset=['APE_SICORE'])  

On a finalement 10.8 millions de liasses.

In [ ]:
DB.shape[0].compute()

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

### 1) Preprocessing 

In [46]:
DB1 = DB[["APE_SICORE","LIB_SICORE"]]
DB1 = DB1.dropna(subset=['LIB_SICORE'])  

In [47]:
stopwords_ = set(stopwords.words('french') + ['a'])
def CleanLib(lib):
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join([x.lower() for x in lib.split() if x.lower() not in stopwords_])

In [48]:
DB1["LIB_CLEAN"] = DB1["LIB_SICORE"].apply(lambda x: CleanLib(x), meta=pd.Series(dtype='str', name='LIB_CLEAN'))
df = DB1.compute()

In [ ]:
df

### 2) Splitting

On mélange de manière aléatoires les index puis on les divise en 2 groupes selon un certain pourcentage (ici 80% et 20%)


In [49]:
random.seed(123456)
Idx = random.sample(df.index.values.tolist(), df.shape[0])
Groups = np.split(Idx, [int(len(Idx)*0.8)])

In [ ]:
with open("../data/train_text.txt", 'w') as f:
    for idx in range(len(Groups[0])):
        aLine = "__label__{} {}".format(df.at[Groups[0][idx],"APE_SICORE"], df.at[Groups[0][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)


### 3) Training

On définit plusieurs options pour le modèle.

In [ ]:
config_fasttext={"dim": 150,
"lr": 0.2,
"epoch": 80,
"wordNgrams": 3,
"minn": 3,
"maxn": 4,
"minCount": 3,
"bucket": 3000000,
"thread": 25,
"loss": 'ova',
"label_prefix": '__label__'}

In [51]:
#model1 = fasttext.train_supervised(input="../data/train_text.txt", **config_fasttext)
model1 = fasttext.load_model("../fasttext_model1.bin")

In [52]:
TypeData = ["Train", "Test"]
dico_lib= { TypeData[i] : {Groups[i][idx] : {"TrueValue" : df.at[Groups[1][idx],"APE_SICORE"],
                                                "Libelle" : df.at[Groups[1][idx],"LIB_CLEAN"]} 
                                                for idx in range(len(Groups[1]))
                            } for i in range(len(TypeData))
}

In [76]:
model1.predict(df_train['LIB_CLEAN'].to_list())

AttributeError: 'numpy.ndarray' object has no attribute 'find'

In [92]:
out = model1.predict(df_train['LIB_CLEAN'].to_list())

In [99]:
df_train = df.loc[Groups[0], :]
df_train[['PREDICTION', 'PROBA']] = get_pred(df_train['LIB_CLEAN'].to_list(), model1)
df_train['GoodPREDICTION'] = df_train['APE_SICORE'] == df_train['PREDICTION']
sum(df_train['GoodPREDICTION'])/df_train.shape[0] * 100

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


ValueError: Columns must be same length as key

In [124]:
def get_preds(libs, mod):
    out = mod.predict(libs)
    pred = [x.replace("__label__","") for i,x in enumerate(list(itertools.chain(*out[0])))]
    prob = out[1]
    return [pred, prob]

In [115]:
def get_pred(lib, mod):
    out = mod.predict(lib)
    pred = out[0][0].replace("__label__","")
    prob = out[1][0]
    return [pred, prob]

In [126]:
pd.DataFrame(get_preds(df_train['LIB_CLEAN'].to_list(), model1), columns=["A", "Z"])

ValueError: 2 columns passed, passed data had 87128 columns

Accuracy of the training set

In [69]:
import swifter

In [114]:
df_train

,APE_SICORE,LIB_SICORE,LIB_CLEAN,PREDICTION,PROBA,GoodPREDICTION
LIA_NUM,,,,,,
C80018017548,7732Z,LOCATION VENTE NEGOCE D ENGINS DE LEVAGE MANUT...,location vente negoce engins levage manutention,__label__7732Z,0.731069,False
G69518206328,7912Z,L ORGANISATION ET VENTE DE VOYAGES DE SEJOURS...,organisation vente voyages sejours tous servic...,__label__7911Z,0.665421,False
C17010634985,4789Z,"VENTE BONBONS, BARBE A PAPA, BALLONS, JOUETS, ...",vente bonbons barbe papa ballons jouets gadget...,__label__4789Z,0.749097,False
G72529491466,4791A,E COMMERCE VENTE EN LIGNE DE TOUS PRODUITS N...,e commerce vente ligne tous produits non regle...,__label__4791A,0.919653,False
C00910025737,5520Z,LOCATION PARA HOTELIERE,location para hoteliere,__label__5510Z,0.430157,False
...,...,...,...,...,...,...
O00070527658,7912Z,7912Z,z,__label__0111Z,0.060097,False
C42014009270,5510Z,"AUBERGE, RESTAURATION TRADITIONNELLE, BAR, DEP...",auberge restauration traditionnelle bar depot ...,__label__5610A,0.912446,False
G92019614326,6820B,L ACQUISITION L ADMINISTRATION ET L EXPLOITAT...,acquisition administration exploitation bail l...,__label__6820B,0.991948,False


In [119]:
df_train = df.loc[Groups[0], :]
df_train[['PREDICTION', 'PROBA']] = df_train['LIB_CLEAN'].swifter.apply(lambda x: get_pred(x, model1)).to_list()
df_train['GoodPREDICTION'] = df_train['APE_SICORE'] == df_train['PREDICTION']
sum(df_train['GoodPREDICTION'])/df_train.shape[0] * 100

Pandas Apply:   0%|          | 0/87128 [00:00<?, ?it/s]

84.77068221467266

Accuracy of the testing set

In [70]:
df_test = df.loc[Groups[1], :]
df_test[['PREDICTION', 'PROBA']] = df_test['LIB_CLEAN'].swifter.apply(lambda x: get_pred(x, model1)).to_list()
df_test['GoodPREDICTION'] = df_test['APE_SICORE'] == df_test['PREDICTION']
sum(df_test['GoodPREDICTION'])/df_test.shape[0] * 100

Pandas Apply:   0%|          | 0/21783 [00:00<?, ?it/s]

85.27750998485058

In [ ]:
df_naf =  pd.read_csv(r"../data/naf_extended.csv",sep=",",encoding="utf-8",dtype=str)
df_naf["NIV5"] = df_naf["NIV5"].str.replace(".","")        
df_naf.set_index("NIV5", inplace=True)

In [ ]:
df_naf = df_naf[["NIV1","NIV2", "NIV3", "NIV4", "LIB_NIV5"]]

On regarde la precision le rappel et le F1 pour les différentes classes désaggrégées.

In [ ]:
df_naf = df_naf.join(df_train.groupby(['APE_SICORE']).size().rename("Size_TRAIN").to_frame())
df_naf = df_naf.join(df_test.groupby(['APE_SICORE']).size().rename("Size_TEST").to_frame())
df_naf = df_naf.join(df_test.groupby(['APE_SICORE']).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Recall_TEST"}))
df_naf = df_naf.join(df_test.groupby(['PREDICTION']).mean('GoodPREDICTION').rename(columns={"GoodPREDICTION":"Precision_TEST"}))
df_naf = df_naf.join(2 * 1/(1/df_naf["Precision_TEST"] + 1/df_naf["Recall_TEST"]).rename("F1_TEST").to_frame())

On fait la même chose au niveau moins aggrégé